In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,332 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [44.6 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/un

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [4]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home_sales')

result = spark.sql("SELECT * FROM home_sales")
result.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query1 = """
    SELECT
        YEAR(date) AS Year,
        ROUND(AVG(price), 2) AS Avg_Price
    FROM
        home_sales
    WHERE
        bedrooms = 4
    GROUP BY
        YEAR(date)
    ORDER BY
        YEAR(date)
"""

# Execute the query
result = spark.sql(query1)

# Show the result
result.show()


+----+---------+
|Year|Avg_Price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query2 = """
    SELECT
        date_built AS Year,
        ROUND(AVG(price), 2) AS Avg_Price
    FROM
        home_sales
    WHERE
        bedrooms = 3
        AND bathrooms = 3
    GROUP BY
        date_built
    ORDER BY
        date_built
"""

# Execute the query
result = spark.sql(query2)

# Show the result
result.show()


+----+---------+
|Year|Avg_Price|
+----+---------+
|2010|292859.62|
|2011|291117.47|
|2012|293683.19|
|2013|295962.27|
|2014|290852.27|
|2015| 288770.3|
|2016|290555.07|
|2017|292676.79|
+----+---------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query3 = """
    SELECT
        date_built AS Year,
        ROUND(AVG(price), 2) AS Avg_Price
    FROM
        home_sales
    WHERE
        bedrooms = 3
        AND bathrooms = 3
        AND floors = 2
        AND sqft_living >= 2000
    GROUP BY
        date_built
    ORDER BY
        date_built
"""

# Execute the query
result = spark.sql(query3)

# Show the result
result.show()

+----+---------+
|Year|Avg_Price|
+----+---------+
|2010|285010.22|
|2011|276553.81|
|2012|307539.97|
|2013|303676.79|
|2014|298264.72|
|2015|297609.97|
|2016| 293965.1|
|2017|280317.58|
+----+---------+



In [20]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

# Record the start time
start_time = time.time()

# Spark SQL query to calculate average price for homes greater than or equal to $350,000
query = """
    SELECT
        AVG(price) AS Avg_Price
    FROM
        home_sales
    WHERE
        price >= 350000
"""

# Execute the query
result = spark.sql(query)

# Show the result
result.show()

# Record the end time
end_time = time.time()

# Print the runtime
print("--- %s seconds ---" % (end_time - start_time))

+------------------+
|         Avg_Price|
+------------------+
|473796.26220224716|
+------------------+

--- 0.7053132057189941 seconds ---


In [15]:
# 7. Cache the the temporary table home_sales.
home_sales.cache()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [17]:
# 8. Check if the table is cached.
is_cached = spark.catalog.isCached('home_sales')
print(is_cached)

False


In [21]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Record the start time
start_time_cached = time.time()

# Spark SQL query to filter out view ratings with average price greater than or equal to $350,000
query_cached = """
    SELECT
        AVG(Price) AS Avg_Price
    FROM
        home_sales
    WHERE
        Price < 350000
"""

# Execute the query using the cached data
result_cached = spark.sql(query_cached)

# Show the result
result_cached.show()

# Record the end time
end_time_cached = time.time()

# Print the runtime for the cached query
print("--- Cached Query Runtime: %s seconds ---" % (end_time_cached - start_time_cached))

+------------------+
|         Avg_Price|
+------------------+
|235148.14186445266|
+------------------+

--- Cached Query Runtime: 0.5010735988616943 seconds ---


In [23]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales.write.partitionBy("date_built").parquet("/path/to/output")

In [25]:
# 11. Read the parquet formatted data.
parquet_path = "/path/to/output"

# Read the Parquet data into a DataFrame
read_df = spark.read.parquet(parquet_path)

# Show the contents of the DataFrame
read_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [26]:
# 12. Create a temporary table for the parquet data.
read_df.createOrReplaceTempView('parquet_temp')

In [28]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time_parquet = time.time()
# Spark SQL query to filter out view ratings with average price greater than or equal to $350,000
query_parquet = """
    SELECT
        AVG(Price) AS Avg_Price
    FROM
        parquet_temp
    WHERE
        Price < 350000
"""

# Execute the query using the Parquet DataFrame
result_parquet = spark.sql(query_parquet)

# Show the result
result_parquet.show()

# Record the end time for Parquet query
end_time_parquet = time.time()

# Print the runtime for the Parquet query
print("--- Parquet Query Runtime: %s seconds ---" % (end_time_parquet - start_time_parquet))

+------------------+
|         Avg_Price|
+------------------+
|235148.14186445266|
+------------------+

--- Parquet Query Runtime: 0.35502028465270996 seconds ---


In [34]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home_sales')

In [35]:
# 15. Check if the home_sales is no longer cached
is_cached_after_uncache = spark.catalog.isCached('home_sales')
print(is_cached_after_uncache)


False
